In [1]:
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import pyLDAvis.sklearn
from nltk.stem.snowball import SnowballStemmer

In [2]:
data = pd.read_json('articles.json', orient='index')['text'].values

In [3]:
stemmer = SnowballStemmer("english")
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])

In [4]:
# tokenize, remove stopwords, lowercase, and ignore words 
# that happen too much or too little
vectorizer = StemmedCountVectorizer(
    min_df=0.02, 
    max_df=0.80, 
    stop_words='english', 
    lowercase=True,
    analyzer="word",
    token_pattern='[a-zA-Z\-0-9][a-zA-Z\-0-9]{2,}'
)
data_vectorized = vectorizer.fit_transform(data)
 
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(
    n_components=10,
    max_iter=50, 
    learning_method='online'
)
lda_Z = lda_model.fit_transform(data_vectorized)

In [5]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(
    lda_model, 
    data_vectorized,
    vectorizer, 
    mds='tsne'
)
panel

/usr/local/var/pyenv/versions/2.7.10/lib/python2.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
8      24.035322        1       1  135.316162  -81.823463
0      13.863055        1       2   88.672409  136.363541
9      13.423366        1       3 -108.075592   44.130524
3      12.047260        1       4  -84.763084  -77.364761
5      11.078948        1       5   17.091536 -115.697395
2       7.667545        1       6  -27.159990   -3.634840
6       6.493959        1       7  142.009521   34.870022
1       5.967357        1       8  -33.990238  122.362404
7       3.683523        1       9   60.091885  -28.783564
4       1.739665        1      10   38.142303   59.227020, topic_info=     Category         Freq         Term        Total  loglift  logprob
term                                                                  
234   Default  2261.000000           aw  2261.000000  30.0000  30.0000
1457  Default  4065.000000        learn  4065.000000  29.0000  29.0000
128   Default  1471.000000       amazon  1471.000000  28.0000  28.0000
666   Default  1690.000000         deep  1690.000000  27.0000  27.0000
2063  Default  1174.000000     reinforc  1174.000000  26.0000  26.0000
635   Default  3757.000000         data  3757.000000  25.0000  25.0000
1635  Default  1569.000000        model  1569.000000  24.0000  24.0000
2244  Default   947.000000       servic   947.000000  23.0000  23.0000
1683  Default  1130.000000      network  1130.000000  22.0000  22.0000
2605  Default   809.000000         type   809.000000  21.0000  21.0000
983   Default   791.000000       featur   791.000000  20.0000  20.0000
1103  Default  1101.000000       github  1101.000000  19.0000  19.0000
2188  Default   855.000000       scienc   855.000000  18.0000  18.0000
1549  Default   827.000000        manag   827.000000  17.0000  17.0000
1275  Default   839.000000       import   839.000000  16.0000  16.0000
1534  Default  1249.000000       machin  1249.000000  15.0000  15.0000
2580  Default   415.000000         tree   415.000000  14.0000  14.0000
1696  Default   490.000000         node   490.000000  13.0000  13.0000
1003  Default   406.000000       filter   406.000000  12.0000  12.0000
117   Default   695.000000    algorithm   695.000000  11.0000  11.0000
1989  Default  1198.000000       python  1198.000000  10.0000  10.0000
2658  Default   928.000000         valu   928.000000   9.0000   9.0000
1947  Default   707.000000      process   707.000000   8.0000   8.0000
1727  Default   246.000000          obj   246.000000   7.0000   7.0000
26    Default   957.000000         2018   957.000000   6.0000   6.0000
1606  Default   687.000000       method   687.000000   5.0000   5.0000
481   Default   308.000000       column   308.000000   4.0000   4.0000
175   Default   446.000000  architectur   446.000000   3.0000   3.0000
853   Default   218.000000       endobj   218.000000   2.0000   2.0000
2568  Default   880.000000        train   880.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
5     Topic10    25.623596          101    45.559796   3.4760  -5.3374
238   Topic10    27.420526          axi    50.565459   3.4395  -5.2696
2558  Topic10     7.940398    traceback    14.742725   3.4327  -6.5090
2733  Topic10    35.368949        width    67.576578   3.4041  -5.0151
639   Topic10    40.692606     datafram    81.667226   3.3549  -4.8749
481   Topic10   152.541818       column   308.224982   3.3481  -3.5535
7     Topic10     6.486447          103    13.731682   3.3015  -6.7112
1025  Topic10    41.230477        float    89.574592   3.2756  -4.8618
2574  Topic10    17.605878     transpar    38.462928   3.2700  -5.7127
1263  Topic10    54.871762      illustr   130.759474   3.1831  -4.5759
1639  Topic10    73.015644        modul   175.169505   3.1764  -4.2903
2400  Topic10    98.189799       stream   235.662294   3.1760  -3.9940
694   Topic10    16.451105      descent    41.541778   3.1252  -5.